In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,SGD

from tensorflow.keras.datasets import cifar10
import os
import tensorflow.keras as K
import tensorflow as tf

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.0
Running on TPU  ['10.86.205.42:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.86.205.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.86.205.42:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

batch_size = 16 * tpu_strategy.num_replicas_in_sync

# batch_size = 32

In [ ]:
shape = (331, 331, 3)
target_size = (331, 331)

input_tensor = K.Input(shape=shape)
adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.9999, decay=0.0)


def create_model():        
    model_base = K.applications.ResNet101(include_top=False, weights="imagenet", input_tensor=input_tensor,input_shape=shape)
    
    #model_base.trainable = False

    model = K.models.Sequential(name='my_model')
    model.add(K.layers.Lambda(lambda image: tf.image.resize(image, target_size)))

    # augmentations = tf.keras.Sequential([
    #   tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    #   tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.8),
    #   tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.1, dtype=tf.float32),
    #   # tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1,width_factor=0.1, dtype=tf.float32),
    #   ],name='augmentations')
    
    # model.add(augmentations)

    model.add(model_base)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(10, activation='softmax'))  
    
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics = ['accuracy'])

    return model

with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = create_model()
  model.build((None, 331, 331, 3))
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_2 (Lambda)           (None, 331, 331, 3)       0         
                                                                 
 NASNet (Functional)         (None, 11, 11, 4032)      84916818  
                                                                 
 flatten_2 (Flatten)         (None, 487872)            0         
                                                                 
 dropout_2 (Dropout)         (None, 487872)            0         
                                                                 
 batch_normalization_2 (Batc  (None, 487872)           1951488   
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 10)                4878730   
                                                          

In [ ]:
def preprocess_data(X, Y):

    X = K.applications.resnet50.preprocess_input(X)
    # X = X / 255
    Y = K.utils.to_categorical(Y, 10)
    
    return X, Y

In [ ]:
(X_train, y_train), (X_test, y_test) = K.datasets.cifar10.load_data()
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)


checkPoint=ModelCheckpoint("./model.h5", save_weights_only=False, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
earlyStopping=EarlyStopping(monitor='val_loss', min_delta = 0.01, patience=7, verbose=0, mode='min') 
lrr = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=2.5e-5)

In [ ]:
history = model.fit(X_train,y_train,validation_split=0.2 ,
                epochs=30,
                batch_size=batch_size,
                callbacks = [lrr,checkPoint,earlyStopping],
                verbose=2)

Epoch 1/30

Epoch 1: val_accuracy improved from -inf to 0.88820, saving model to ./model.h5
313/313 - 260s - loss: 0.4334 - accuracy: 0.8664 - val_loss: 0.5006 - val_accuracy: 0.8882 - lr: 1.0000e-04 - 260s/epoch - 830ms/step
Epoch 2/30

Epoch 2: val_accuracy did not improve from 0.88820
313/313 - 73s - loss: 0.1016 - accuracy: 0.9664 - val_loss: 0.7294 - val_accuracy: 0.8816 - lr: 1.0000e-04 - 73s/epoch - 233ms/step
Epoch 3/30

Epoch 3: val_accuracy did not improve from 0.88820
313/313 - 73s - loss: 0.0510 - accuracy: 0.9841 - val_loss: 0.8329 - val_accuracy: 0.8808 - lr: 1.0000e-04 - 73s/epoch - 233ms/step
Epoch 4/30

Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 4: val_accuracy did not improve from 0.88820
313/313 - 73s - loss: 0.0288 - accuracy: 0.9923 - val_loss: 0.8874 - val_accuracy: 0.8797 - lr: 1.0000e-04 - 73s/epoch - 234ms/step
Epoch 5/30

Epoch 5: val_accuracy did not improve from 0.88820
313/313 - 73s - loss: 0.0191 - accuracy: 0.9954 -

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

index = np.argmax(history.history['val_accuracy'])

print('last model accuracy',history.history['val_accuracy'][-1])
print('best model accuracy',history.history['val_accuracy'][index])

model = tf.keras.models.load_model('./model.h5') 
print('model accuracy on test data:',model.evaluate(X_test, y_test, batch_size=batch_size))


In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
import pandas as pd
import seaborn as sns
import tensorflow.keras as K

labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

predictions = model.predict(X_test)

In [ ]:


df = pd.DataFrame({'predictions':np.argmax(predictions,axis=1),'actual_values':np.argmax(y_test,axis=1)})

df['predictions']=df['predictions'].apply(lambda x: labels[x] )
df['actual_values']=df['actual_values'].apply(lambda x: labels[x] ) 

expected = df['predictions'].to_numpy()
actual = df['actual_values'].to_numpy()

# print(expected[:10])
# print(actual[:10])

cm=confusion_matrix(actual,expected)

plt.figure(figsize=[15, 10])
sns.heatmap(data=cm, annot=True, cmap='Blues', square=True, xticklabels=labels, yticklabels=labels)
plt.xlabel('Actual label')
plt.ylabel('Expected label')
plt.show()

print( classification_report(actual,expected))

display(df.head())

failings = df[df['predictions']!=df['actual_values']]
corrects = df[df['predictions']==df['actual_values']]

display(failings.head())

failing_predictions = len(failings)

print('n° of failing predictions:',failing_predictions)
# print(failings.index[0])
total_data = len(df)

print('accuracy:',1 - failing_predictions / total_data)
# print('accuracy:',model.evaluate(X_test, y_test, batch_size=256))



fig = plt.figure(figsize=(20, 20))

columns = 5
rows = 10
total_pics = columns*rows
# print(failings.iloc[failings.index[0]])

(X_train, y_train), (X_test, y_test) = K.datasets.cifar10.load_data()
for i in range(total_pics):
  if(i <total_pics):
    ax = fig.add_subplot(rows, columns, i+1)
  el = failings.iloc[i]
  ax.set_title('expecting :'+str(el.actual_values)+", predicted: "+str(el.predictions),fontsize=12)
  
  plt.imshow(X_test[failings.index[i]])
  plt.axis('off')
plt.tight_layout()

fig = plt.figure(figsize=(20, 20))

columns = 5
rows = 10
total_pics = columns*rows

for i in range(total_pics):
  if(i <total_pics):
    ax = fig.add_subplot(rows, columns, i+1)
  el = corrects.iloc[i]
  ax.set_title("predicted: "+str(el.predictions),fontsize=12)
  
  plt.imshow(X_test[corrects.index[i]])
  plt.axis('off')
plt.tight_layout()

In [3]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,SGD